In [1]:
#import jax and other libraries for computation
import jax.numpy as jnp
from jax import jit
from jax.scipy.signal import convolve2d
from jax.flatten_util import ravel_pytree
from jax.experimental.ode import odeint
from jax import tree_util
import jax.random as random
import numpy as np
#for visulization
import os
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
# Set Palatino as the default font
font = {'family': 'serif', 'serif': ['Palatino'], 'size': 20}
plt.rc('font', **font)
plt.rc('text', usetex=True)
# import AdoptODE
from adoptODE import train_adoptODE, simple_simulation, dataset_adoptODE
#import the MSD mechanics
from HelperAndMechanics import *
import h5py

In [2]:
# Load from HDF5
with h5py.File('../data/SpringMassModel/MechanicalData/data_eta_05_uvx.h5', 'r') as f:
    v = f['v'][:1000]
    u = f['u'][:1000]
    T = f['T'][:1000]
    x = f['x'][:1000]
    f.close()
print('max_v = ',np.max(v),'\nmax_u =',np.max(u),' \nmax_T = ',np.max(T))
N = T.shape[0]


max_v =  2.14554 
max_u = 0.988905  
max_T =  2.71647


In [ ]:
def define_NavierCauchy(**kwargs_sys):
    N_sys = kwargs_sys['N_sys']

    def gen_params():
        return {key:value + kwargs_sys['par_tol']*value*np.random.uniform(-1.0, 1.0) for key,value in kwargs_sys['params_true'].items()}, {}, {}
    
    def gen_y0():
        return {'u':kwargs_sys['u0'],'v':kwargs_sys['v0'],'T':kwargs_sys['T0'],'U':kwargs_sys['U0'],'V':kwargs_sys['V0']}
    
    # electric model
    @jit
    def kernel(spacing):
        kernel = np.array([[1, 4, 1], [4, -20.0, 4], [1, 4, 1]]) / (spacing* spacing * 6)
        return kernel
    @jit
    def laplace(f,spacing):  #laplace of scalar
        f_ext = jnp.concatenate((f[0:1], f, f[-1:]), axis=0)
        f_ext = jnp.concatenate((f_ext[:, 0:1], f_ext, f_ext[:, -1:]), axis=1)
        return convolve2d(f_ext, kernel(spacing), mode='valid')
    @jit
    def epsilon(u,v,params):
        return params['epsilon_0']+params['mu_1']*v/(u+params['mu_2'])
    @jit
    def epsilon_T(u):
        return 1 - 0.9*jnp.exp(-jnp.exp(-30*(jnp.abs(u) - 0.1)))    

    @jit
    def gradient(field, dx):
        """Compute the gradient of a scalar field field(x, y) in 2D."""
        grad_x = jnp.gradient(field, axis=0) / dx  
        grad_y = jnp.gradient(field, axis=1) / dx  
        return jnp.stack([grad_x, grad_y], axis=-1)  # Shape: (N, N, 2)
    # Define Laplacian operator
    def laplacian(U,dx):
        return (
            -4 * U
            + jnp.roll(U, shift=1, axis=0) + jnp.roll(U, shift=-1, axis=0)
            + jnp.roll(U, shift=1, axis=1) + jnp.roll(U, shift=-1, axis=1)
        ) / dx**2

    # Compute divergence
    def divergence(U,dx):
        return jnp.gradient(U[..., 0], axis=0)/dx + jnp.gradient(U[..., 1], axis=1)/dx

    # Compute stress tensor
    def stress(U, params):
        div_U = divergence(U,params['dx'])
        return (params['lmbda'] + params['mu']) * gradient(div_U,params['dx']) + params['mu'] * laplacian(U,params['dx'])
    #Full Model
    @jit
    def eom(y, t, params, iparams, exparams):
        par=params
        u=y['u']
        v=y['v']
        T=y['T']
        U=y['U']
        V=y['V']

        dudt = par['D']*laplace(u,par['spacing'])-(par['k'])*u*(u-par['a'])*(u-1) - u*v
        dvdt = epsilon(u,v,par)*(-v-(par['k'])*u*(u-par['a']-1))
        dTdt = epsilon_T(u)*(par['k_T']*jnp.abs(u)-T)
        dU = V 
        dV = (1 / par['rho']) * (stress(U,par) + gradient(T,par['dx']))  # Update velocity
        
        return {'u':dudt, 'v':dvdt, 'T':dTdt, 'U':zero_out_edgesFD(dU), 'V':zero_out_edgesFD(dV)}
    @jit
    def loss(ys, params, iparams, exparams, targets):
        # u = ys['u']
        # u_target = targets['u']
        U = ys['U']
        U_target = targets['U']
        V = ys['V']
        V_target = targets['V']
        
        return  jnp.nanmean((U - U_target)**2 + (V-V_target)**2) #jnp.nanmean((u - u_target)**2) +
            
    return eom, loss, gen_params, gen_y0, {}


In [22]:

"""
    Reads in necessary parameters from config.ini
"""

N,size,params_electric = read_config(['D','a','k','epsilon_0','mu_1','mu_2','k_T','delta_t_e','spacing'],mode = 'chaos')
keys_electric =['D','a','k','epsilon_0','mu_1','mu_2','k_T','delta_t_e','spacing']

params_true_mechanic = {'lmbda': 1.25, 'mu': 1, 'rho': 1.0, 'dx': 1.0}
tol = 0
params_true = dict(zip(keys_electric,params_electric))|params_true_mechanic
print(params_true['k_T'])

params_low = {key: value - value*tol for key, value in params_true.items()}
params_high = {key: value + value*tol for key, value in params_true.items()}

length,sampling_rate = 20,10

u0,v0,T0,U0,V0,t_evals = u[0],v[0],T[0],jnp.zeros((100,100,2)),jnp.zeros((100,100,2)),np.linspace(0, params_true['delta_t_e']*sampling_rate*length, length)

kwargs_sys = {'size': 100,
              'spacing': 1,
              'N_sys': 1,
              'par_tol': 0,
              'params_true': params_true,
              'u0': u0,'v0': v0,'T0': T0,'U0': U0,'V0': V0}
kwargs_adoptODE = {'epochs': 10,'N_backups': 1,'lr': 1e-3}

3.0


In [23]:
# Setting up a dataset via simulation
Simulation_MSD = simple_simulation(define_NavierCauchy,
                            t_evals,
                            kwargs_sys,
                            kwargs_adoptODE)

In [ ]:
#plot u_err = np.abs(u_sim-u_sol)
Ux_sim = Simulation_MSD.ys['U'][0, :, :, :, 0]
Uy_sim = Simulation_MSD.ys['U'][0, :, :, :, 1]
u_sim = Simulation_MSD.ys['u'][0]
T_sim = Simulation_MSD.ys['T'][0]
print(Ux_sim.shape)
vmin,vmax =  0,1
pad = 10


# Function to update the plot
def update_plot(frame):
    fig, axes = plt.subplots(1, 2, figsize=(16, 4))  # Create 2 side-by-side subplots
    
    # Plot dA_fit
    im1 = axes[0].matshow(Ux_sim[frame], cmap='coolwarm', vmin=Ux_sim.min(), vmax=Ux_sim.max())
    # im1 = axes[0].matshow(dA_rec[frame, pad:-pad, pad:-pad], cmap='coolwarm', vmin=vmin, vmax=vmax)
    axes[0].set_title(f"reconstruction")
    
    # Plot dA_sim
    im2 = axes[1].matshow(T_sim[frame], cmap='coolwarm', vmin=vmin, vmax=vmax)
    # im2 = axes[1].matshow(dA_sim[frame, pad:-pad, pad:-pad], cmap='coolwarm', vmin=vmin, vmax=vmax)
    axes[1].set_title(f"simulation")
    
    # Add colorbars
    fig.colorbar(im1, ax=axes[0])
    fig.colorbar(im2, ax=axes[1])

    plt.show()
def update_single_plot(frame):
    fig,axes = plt.subplots(1, 1, figsize=(10,8))

    # Plot dA_fit
    im1 = axes.matshow(Ux_sim[frame], cmap='coolwarm', vmin=vmin, vmax=vmax)
    # im1 = axes.matshow(dA_err[frame, pad:-pad, pad:-pad], cmap='coolwarm', vmin=vmin, vmax=vmax)
    axes.set_title(f"U ")

    # Add colorbars
    fig.colorbar(im1, ax=axes)

    plt.show()

# Create interactive slider
frame_slider = widgets.IntSlider(min=0, max=Ux_sim.shape[0]-1, step=1, value=0, description="Frame")

# Use interactive_output instead of interactive
out = widgets.interactive_output(update_plot, {'frame': frame_slider})

# Display slider and output
display(frame_slider, out)

(20, 100, 100)


IntSlider(value=0, description='Frame', max=19)

Output()